# SVM multiclass model with 'rbf' kernel ???
Parameters of the model:
- $C_{\text{raw}} = 10^3$
- $C_{\text{scaled}} = 250$

In [1]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
from src.model.SVM.SVM_module import *
from src.train.metrics import *

In [2]:
data = ManageData()
# Loading the training features and labels
data.LoadTrainFeatures()

Training features and labels loaded!
Example:
X: [-2.03834213e+02  1.65272430e+02 -2.71203651e+01  3.98504753e+01
  1.52355738e+01  1.52752247e+01 -5.82818985e+00  1.24469271e+01
 -2.53585100e+00  5.68161488e+00  2.86470144e-02  4.47464501e-03
  2.23200000e+03  1.20789442e+03  1.52517064e+03  2.32513772e+01
  2.57816840e+01  2.43512780e+01  2.44582548e+01  2.38234885e+01
  2.04741587e+01  2.52746605e+01  6.54243573e-04  6.98350232e-01
  5.18728181e-01  4.41854394e-01  1.96658298e-01  2.66243911e+00] 
y: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
# Loading the validation features and labels
data.LoadValidationFeatures()

Validation features and labels loaded!
Example:
X: [ 4.24629608e+02  1.57892838e+02 -6.61561127e+01  3.53172836e+01
 -1.08259525e+01  1.99779091e+01  9.03547859e+00  1.16253910e+01
 -6.59319687e+00  9.90838528e+00  4.58988983e-02  1.11563884e-01
  2.98343434e+03  1.57618474e+03  1.50766778e+03  2.51165358e+01
  1.80690291e+01  1.96382736e+01  1.99261764e+01  1.70384129e+01
  2.10494327e+01  5.28803644e+01  2.43222667e-03  7.72531224e-01
  6.70360129e-01  6.16755698e-01  3.44813168e-01  2.22349119e+00] 
y: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


## Using non-normalized (raw) features:

In [99]:
# fitting the model
model_raw = multilabelSVM(C=1e3, kernel='rbf')
model_raw.fit(X=data.X_train, y=data.y_train)

Training completed successfully!


multilabelSVM(C=1000.0)

In [100]:
# save the model to disk
filename = '.\src\model\SVM\saved_models\SVM_model_rbf_raw.sav'
pickle.dump(model_raw, open(filename, 'wb'))

In [101]:
# first few predictions of the model
prediction_raw = model_raw.predict(X=data.X_val)
prediction_raw[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

### Calculating various metrics for the given model:

In [102]:
metrics_raw = get_metrics(y_pred=torch.from_numpy(prediction_raw), y_true=torch.from_numpy(data.y_val), return_per_instrument=True)
metrics_raw = pd.Series(metrics_raw)
print(metrics_raw)

accuracy                                        tensor(0.8400)
hamming_distance                                tensor(0.1600)
f1                                              tensor(0.0104)
precision                                       tensor(0.0848)
recall                                          tensor(0.0056)
instruments/cello_accuracy                      tensor(0.9595)
instruments/clarinet_accuracy                   tensor(0.9902)
instruments/flute_accuracy                      tensor(0.9656)
instruments/acoustic_guitar_accuracy            tensor(0.8219)
instruments/electric_guitar_accuracy            tensor(0.7039)
instruments/organ_accuracy                      tensor(0.8882)
instruments/piano_accuracy                      tensor(0.5491)
instruments/saxophone_accuracy                  tensor(0.7948)
instruments/trumpet_accuracy                    tensor(0.9029)
instruments/violin_accuracy                     tensor(0.9287)
instruments/human_voice_accuracy                tensor(

## Using scaled (normalized) features:

In [103]:
# Scaling the data
X_train_scaled = StandardScaler().fit_transform(data.X_train)
X_val_scaled = StandardScaler().fit_transform(data.X_val)

print('Scaled X_train example:\n', X_train_scaled[0], '\nScaled X_val example:\n', X_val_scaled[0])

Scaled X_train example:
 [-2.74572843  0.85348952 -0.2168907   0.59508776  1.36445458  0.40435308
  0.28068384  0.58321878  0.74591997  0.08122014 -0.66495349 -1.40500219
 -0.94628502 -0.97193183 -0.81476227 -0.11382856  1.38710685  0.51018858
  0.53895249  0.68717092  0.22631005 -0.44149987 -0.60745816 -0.79176152
 -0.07275946 -0.77147255 -0.71967014 -0.36199382] 
Scaled X_val example:
 [ 0.46577778  1.14591375 -2.3419525   0.64000228 -0.6452382   1.05630954
  1.98016534  0.62292339  0.52668928  0.36654723  0.17209205  0.38449512
 -0.93832965 -0.79117458 -1.61119927 -0.42495896 -0.60171797 -0.40720068
 -0.1623136  -0.69332528  1.20649317  3.00454542 -0.80527956 -0.68803475
  1.05218384  0.34521036  0.36174927 -0.91609425]


In [104]:
# fitting the model
model_norm = multilabelSVM(C=2.5e2, kernel='rbf')
model_norm.fit(X=X_train_scaled, y=data.y_train)

Training completed successfully!


multilabelSVM(C=250.0)

In [105]:
# save the model to disk
filename = '.\src\model\SVM\saved_models\SVM_model_rbf_norm.sav'
pickle.dump(model_norm, open(filename, 'wb'))

In [106]:
# first few predictions of the model
prediction_norm = model_norm.predict(X=X_val_scaled)
prediction_norm[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

### Calculating various metrics for the given model:

In [107]:
metrics_norm = get_metrics(y_pred=torch.from_numpy(prediction_norm), y_true=torch.from_numpy(data.y_val), return_per_instrument=True)
metrics_norm = pd.Series(metrics_norm)
print(metrics_norm)

accuracy                                        tensor(0.8258)
hamming_distance                                tensor(0.1742)
f1                                              tensor(0.2342)
precision                                       tensor(0.3436)
recall                                          tensor(0.1855)
instruments/cello_accuracy                      tensor(0.9165)
instruments/clarinet_accuracy                   tensor(0.9472)
instruments/flute_accuracy                      tensor(0.9324)
instruments/acoustic_guitar_accuracy            tensor(0.8354)
instruments/electric_guitar_accuracy            tensor(0.7420)
instruments/organ_accuracy                      tensor(0.8636)
instruments/piano_accuracy                      tensor(0.6057)
instruments/saxophone_accuracy                  tensor(0.7432)
instruments/trumpet_accuracy                    tensor(0.8857)
instruments/violin_accuracy                     tensor(0.8550)
instruments/human_voice_accuracy                tensor(